# Decoupling Generalized Linear Model Distribution Choices from Canonical Link Functions


## Sample Data

This data will have an exponentially distributed while the log of the target can be expressed as a linear combination of predictors. In being exponentially distributed, the mean is strictly equal to the standard deviation. This is a different characteristic than seen in a Poisson-distributed target, where the mean is equal to the variance, while the canonical link function is a log-linear model. In fact, the canonical link function of an exponential-distribution response is the Negative Inverse:

$ XB = - \mu ^{-1}$

Here, because the target's log can be linearly expressed, the mean will be expressed as such:

$ XB = ln(\mu) $

In [1]:
import numpy as np
from scipy.optimize import minimize

# Generate some example data
np.random.seed(0)
n_samples = 100
X = np.random.rand(n_samples, 2)  # two predictors
true_beta = np.array([1.0, -2.0])  # true coefficients
true_intercept = 0.5

# Generate Poisson-distributed response variable
mu_ = np.exp(true_intercept + X @ true_beta)
lambda_ = 1/mu_
scale_ = 1/lambda_

y = np.random.exponential(scale_)


In [2]:
print(
    'Mean: ',np.mean(y),
    '\nstd: ',np.std(y),
    '\nvar: ',np.var(y))

Mean:  1.2696557752766635 
std:  1.5442965676889258 
var:  2.384851888975797


## Applying Poisson Optimization to an Exponential Distributed Target with the Log Link

Through data analysis, if the log-linear model is desired due to strong log-linear correlations with predictors, or the advantagous quality of factor decomposition, then one should not simply apply Poisson regression due to the log-linear link function being canonical for Poisson regression without observing the mean and spread characteristics of the target, with respect to distribution parameter lambda. Lambda, the mean and the variance relate in the Poisson distribution through equality:

$ \lambda = \mu = \sigma ^ 2$

If the relationship between the mean and variance shows that the target is Poisson distributed, then Poisson regression would be appropriate. In this exercise, the target belongs to the Exponential distribution, so the variance is not equal to the mean. Poisson regression optimization will estimate values for beta, but they may vary from the true parameter values more than desired.  

Without derivation, the log-likelihood of the Poisson distribution is:  

$ l = -ln(X!) + X ln(\lambda) - \lambda $  

With mean estimate and data:

$ \mu = exp(XB); X = Y $  

Because beta does not appear in the first term, the negative log of a factorial, this conveniently disappears when the derivative in terms of beta is taken. The log-likelihood function to maximize becomes:  

$ l = YXB - exp(XB) $

In [3]:

lambda_ = mu_
#y = np.random.poisson(lambda_)

# Define the negative log-likelihood for Poisson regression
def poisson_neg_log_likelihood(params, X, y):
    intercept = params[0]
    beta = params[1:]
    linear_prediction = intercept + X @ beta
    mu_ = np.exp(linear_prediction)
    lambda_ = mu_
    neg_log_likelihood = -np.sum(y * linear_prediction - lambda_)
    return neg_log_likelihood

# Initial guess for the parameters
initial_params = np.zeros(X.shape[1] + 1)

# Perform the optimization
result = minimize(poisson_neg_log_likelihood, initial_params, args=(X, y), method='L-BFGS-B')

# Extract the estimated parameters
estimated_intercept = result.x[0]
estimated_beta = result.x[1:]

print(f"Estimated intercept: {estimated_intercept}")
print(f"Estimated coefficients: {estimated_beta}")


Estimated intercept: 0.4804326098883521
Estimated coefficients: [ 0.75846516 -1.54795698]


## Applying Exponential Optimization to an Exponential Distributed Target with the Log Link

By analyzing the target, one would observe in this exercise that the mean is more nearly equal to the standard deviation than the variance. This would motivate the modeler to choose the Exponential distribution for optimizing the target through Maximum Likelihood Estimation, where the relationship between distribution parameter lambda, the eman and the variance is:

$ \lambda = \frac{1}{\mu} = \frac{1}{\sigma} \rightarrow \mu = \sigma$  

Without derivation, the log-likelihood of the Exponential distribution is:  

$ l = ln(\lambda) - \lambda X $  

With mean estimate and data:

$ \mu = exp(XB); X = Y $  

The log-likelihood function to maximize becomes:  

$ l = -XB - exp(-XB)Y $

Exponential distribution MLE for solving the regression yields different coefficients than the Poisson optimization solution, as expected, and in this exercise, the intercept varies further from the true intercept, while the predictor coefficients are closer to their true values.

In [4]:

lambda_ = 1/mu_
scale_ = 1/lambda_

#y = np.random.exponential(scale_)

def exponential_neg_log_likelihood(params, X, y):
    intercept = params[0]
    beta = params[1:]
    linear_prediction = intercept + X @ beta
    mu_ = np.exp(linear_prediction)
    lambda_ = 1/mu_
    neg_log_likelihood = -np.sum(-linear_prediction - lambda_ * y)
    return neg_log_likelihood


# Initial guess for the parameters
initial_params = np.zeros(X.shape[1] + 1)

# Perform the optimization
result = minimize(exponential_neg_log_likelihood, initial_params, args=(X, y), method='L-BFGS-B')

# Extract the estimated parameters
estimated_intercept = result.x[0]
estimated_beta = result.x[1:]

print(f"Estimated intercept: {estimated_intercept}")
print(f"Estimated coefficients: {estimated_beta}")



Estimated intercept: 0.448432417057888
Estimated coefficients: [ 0.87256177 -1.62201764]


## Applying Least-Squares (Gaussian assumptions) to an Exponential Distributed Target with the Log Link

A very common technique in machine learning optimization is to minimize the Sum of Squared errors in order to solve for beta. The SSE cost function has the advantage (for the identity link - traditional linear regression) of an analytical solution. The relationship between the sum of squared errors and the Gaussian distribution is that Gaussian maximum likelihood estimation is a derivation of the sum of squared errors cost function - they are indelibly linked.

$ L = (2 \pi \sigma ^ 2)^{\frac{1}{2}} exp( -\frac{1}{2 \sigma ^2} (X - \mu)^2 )$

$ l = - \frac{1}{2}ln(2 \pi) - \frac{1}{2}ln( \sigma ^2 ) - \frac{1}{2 \sigma ^ 2}(X - \mu)^2$

with 

$ \mu = exp(XB)$

Optimizing beta to maximize the log-likelihood function is a matter of differentiating in terms of beta, which only appears in the third term. 

$ max_B (l) = max_B (- \frac{1}{2 \sigma ^ 2}(X - \mu)^2) = max_B (- \frac{1}{2}(X - \mu)^2) $  

This is the sum of squared errors cost function, derived from maximum likelihood estimation on the Gaussian distribution. 

With mean estimate and data:

$ \mu = exp(XB); X = Y $  

The log-likelihood function to maximize becomes:  

$ l = - \frac{1}{2}ln( \sigma ^2 ) - \frac{1}{2 \sigma ^ 2}(Y - exp(XB))^2 $

If the target is Gaussian, this technique for optimization will be appropriate, but if the target is exponentially distributed, this technique will yield coefficients that are farther from true values. Here, optimization converges, but the predictor coefficients are farther from the true values than in either Poisson or Exponential regression. The intercept remains in the region of the true value. 



In [5]:
#Equivalent SSE function, not estimating sigma
def sse_neg_log_likelihood(params, X, y):
    intercept = params[0]
    beta = params[1:-1]
    linear_prediction = intercept + X @ beta
    mu_ = np.exp(linear_prediction)
    residuals = y-mu_
    neg_log_likelihood = -np.sum(-.5 * ((residuals) ** 2))
    return neg_log_likelihood

# Define the negative log-likelihood for Gaussian regression
def gaussian_neg_log_likelihood(params, X, y):
    intercept = params[0]
    beta = params[1:-1]
    sigma=params[-1]
    linear_prediction = intercept + X @ beta
    mu_ = np.exp(linear_prediction)
    residuals = y - mu_
    neg_log_likelihood = np.sum(.5*np.log(sigma**2) + .5*(residuals**2) / (sigma**2))
    return neg_log_likelihood


# Initial guess for the parameters
initial_params = np.concatenate([np.zeros(X.shape[1] + 1),np.ones(1)])

# Perform the optimization
result = minimize(gaussian_neg_log_likelihood, initial_params, args=(X, y), method='L-BFGS-B')

# Extract the estimated parameters
estimated_intercept = result.x[0]
estimated_beta = result.x[1:-1]
estimated_sigma = result.x[-1]

print(f"Estimated intercept: {estimated_intercept}")
print(f"Estimated coefficients: {estimated_beta}")
print(f"Estimated sigma: {estimated_sigma}")



Estimated intercept: 0.5483205461701764
Estimated coefficients: [ 0.60481991 -1.47466755]
Estimated sigma: 1.4234723049616829
